<a href="https://colab.research.google.com/github/Chaitrahooli/Machine-Learning-Project/blob/master/Contextual_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
files.upload()

Saving intents4.json to intents4.json


{'intents4.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "mopeds",\r\n         "patterns": [

# Contextual chatbot
We are going to create the chatbot framework and build the conversational model for the island moped rental shop.The chatbot for this small business needs to handle the simple questions about hours of operations and reservation options and so on.We also want it to handle the contextual responses such as inquires about the same day rental.



*   First we will transform the conservational intent defnition to the TensorFlow model
*   Next,we will build the chatbot framework to process the response.

*   Lastly we will show how the basic context can be incorporated into our responses


In chatbot framework we need to define the conservation intent,which is like JSON file.












Here we need to import the some important laibraries for NLP, here we pass the tflearn layer for the TensorFlow model.

In [3]:
#things we need for nltk
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

W0819 08:39:52.974696 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0819 08:39:52.976638 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0819 08:39:52.994761 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0819 08:39:53.002831 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0819 08:39:53.017735 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/

Here first we need to load the conversational intent json file.

In [0]:
#import our intents file
import json
with open('./intents4.json') as json_data:
  intents=json.load(json_data)


we need to download the **punkt** nltk package used for wordtokenisation

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Each conversational intent contains the 


*   **tag**(unique name)
*   **patterns**(sentence patterns for our nueral network text classifier)


*   **responses**(giving the responses what we want)

Here we are doing some preprocesiing steps for nlp like tokenising, stemming  etc.for the given intent and saparate these intents as a words classes and documents as shown below here we ignore the character **'?'**





In [6]:
words=[]
classes=[]
document=[]
ignore_word=['?']
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the sentence
    w=nltk.word_tokenize(pattern)
    #append to the words list
    words.extend(w)
    #add to the document
    document.append((w,intent['tag']))
    #add to the classes
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
words=[stemmer.stem(w.lower())for w in words if w not in ignore_word]
words=sorted(list(set(words)))

#remove duplicates
classes=sorted(list(set(classes)))
print(len(document),"documents",document)
print('#################################################')
print(len(classes),"classes",classes)
print("#################################################")
print(len(words),"words",words)

27 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'), (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'), (['What', 'do', 'you', 'rent', '?'], 'mopeds'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['Are', 'you', 'open', 'today', '?'], 'opentoday'), (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'), (['What', 'are', 'your', 'hours',

The resultant out put should contains the 27 documents,9 classes, 48 words

Here we create the training data and bag of words for each classes. 

In [7]:
#create the training data
training=[]
output=[]
total_data=[]
#create empty array for output
output_empty=[0]*len(classes)

#training set bag of words for each classes
for doc in document:
  #initialize the bag of words
  bag=[]
  #list of tokenized words for the patterns
  pattern_words=doc[0]
  print('-------------->',pattern_words)
  
  #stemm for each word
  pattern_words=[stemmer.stem(word.lower())for word in pattern_words]
  #create our bag of words
  for w in words:
    bag.append(1)if w in pattern_words else bag.append(0)
  #output is a 0 for each tag and 1 for current tag
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
 # print(output_row)
 # print('#################################################')
  total_data.append(output_row)
  training.append([bag,output_row])
  print(training)
 # print('##################################')
print(len(training))  

--------------> ['Hi']
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]]]
--------------> ['How', 'are', 'you']
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0]]]
--------------> ['Is', 'anyone', 'there', '?']
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0,

In [8]:
import pandas as pd
pd.DataFrame(total_data,columns=classes)

,goodbye,greeting,hours,mopeds,opentoday,payments,rental,thanks,today
0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,1,0


In [9]:
#shuffle our features abd turn into np.array
random.shuffle(training)
training=np.array(training)
#print(training)
#print('#############################')
#create the train and test list
X_train=list(training[:,0])
#print(X_train)
#print('#####################################################')
y_train=list(training[:,1])
print(y_train)

[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0]]


In [10]:
#reset underlying graph data
tf.reset_default_graph()
net=tflearn.input_data(shape=[None,len(X_train[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(y_train[0]),activation='softmax')
net=tflearn.regression(net)

W0819 08:40:26.237750 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 08:40:26.242951 140714809804672 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tflearn/initializations.py:174: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 08:40:26.302518 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/optimizers.py:238: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0819 08:40:26.317086 140714809804672 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tflearn/objectives.py:66: calling reduce_sum_v1 (

In [11]:
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

W0819 08:40:29.885862 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/summaries.py:46: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0819 08:40:29.966152 140714809804672 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0819 08:40:30.119888 140714809804672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:134: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



In [12]:
#define model and setup tensorboard

#start training and apply gradient descent algorithm
model.fit(X_train,y_train, n_epoch=3000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Training Step: 5999  | total loss: 0.51087 | time: 0.004s
| Adam | epoch: 3000 | loss: 0.51087 - acc: 0.9143 -- iter: 16/27
Training Step: 6000  | total loss: 0.47644 | time: 0.007s
| Adam | epoch: 3000 | loss: 0.47644 - acc: 0.9228 -- iter: 27/27
--


In [0]:
def clean_up_sentence(sentence):
  sentence_word=nltk.word_tokenize(sentence)
  sentence_word=[stemmer.stem(word.lower())for word in sentence_word]
  return sentence_word

In [0]:
#return the bag of word array:0 or 1 for each word in the bag that exists in the sentence
def bow(sentence,words,show_details=False):
  #tokenize patterns
  sentence_word=clean_up_sentence(sentence)
  #bag of words
  bag=[0]*len(words)
  for s in sentence_word:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
      if show_details:
        print("found in bag:%s" %w)
  return(np.array(bag))

In [15]:
p=bow('Is your shop open today?',words)
df_data=[]
df_data.append(p)
pd.DataFrame(df_data,columns=words)

,'d,'s,a,acceiv,anyon,ar,bye,can,card,cash,credit,day,do,doe,good,goodby,hav,hello,help,hi,hour,how,i,is,kind,lat,lik,mastercard,mop,of,on,op,rent,see,tak,thank,that,ther,thi,to,today,we,what,when,which,work,yo,you
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0


# Response Processor

In [0]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    ##print("results",results)
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

In [0]:

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [18]:
classify('do i need to bring cash for shopping')

[('payments', 0.57489836), ('rental', 0.27158323)]

In [19]:
response('do i need to bring cash for shopping?')

We accept VISA, Mastercard and AMEX


In [20]:
classify('is your shop open today?')

[('opentoday', 0.6436566)]

In [21]:
response('is your shop open today?')

Our hours are 9am-9pm every day


In [22]:
response('what kind of mopeds do you rent?')

We rent Yamaha, Piaggio and Vespa mopeds


In [23]:
response('goodbye,see you later')

Have a nice day


In [24]:
response('hi,how are you?')

Good to see you again


In [25]:
response('I am very happy to see you')

Good to see you again


# Contextualisation

In [0]:
context={}
ERROR_TRESHOLD=0.25
def classify(sentence):
  #generate the probabilities from the model
  results=model.predict([bow(sentence,words)])[0]
  #filter out predictions below threshold
  results=[[i,r] for i,r in enumerate(results) if r>ERROR_TRESHOLD]
  #sort by strength of probability
  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append((classes[r[0]],r[1]))
  return return_list

In [0]:

#create a datastructure to hold the user context
def response(sentence,USERID='123',show_details=False):
  results=classify(sentence)
  #if we have a classification then find the matching intent
  if results:
    #loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        #find the tag matching the first intent
         if i['tag']==results[0][0]:
            #Set the context for this if necessary
            if 'context_set' in i:
              if show_details:
                print('context:',i['context_set'])
              context[USERID]=i['context_set']
              
              
            #cheak if this intent is contextual and apply to this users conversations
            if not 'context_filter' in i or \
              (USERID in context and 'context_filter' in i and i['context_filter']==context[USERID] ):
              if show_details:print('tag:',i['tag'])
              #random response from the intent
              return print(random.choice(i['responses']))
            
            
      results.pop(0)
                
  

In [28]:
response('we want to moped the rent')

Are you looking to rent today or later this week?


In [29]:
response('today')

For rentals today please call 1-800-MYMOPED
